In [2]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Financial spendings dataset
**We are showing the trends of spending per month in the form of bar charts.**

# Data Inspection - unpreprocessed dataset

In [24]:
#Reading in the CSV
os.getcwd()
file_path01 = r"../data/cristina_dataset/Cristina_Dataset_230316.csv"
df = pd.read_csv(file_path01)

In [25]:
#Looking at the dataset
df

,Category,Detail,Cost,Date
0,Eating Out,Brunch,€15.00,25-Jul-2022
1,Shopping,Uniqlo,€144.50,25-Jul-2022
2,Hobbies,Salsa Class,€130.00,27-Jul-2022
3,Groceries,Groceries,€6.78,27-Jul-2022
4,Eating Out,Ditsch,€2.00,27-Jul-2022
...,...,...,...,...
324,Groceries,Groceries,€8.26,21-Feb-2023
325,Groceries,Groceries,€27.25,22-Feb-2023
326,Groceries,Groceries,€13.69,23-Feb-2023
327,Rent + Bills,Phone Norway,€4.50,24-Feb-2023


In [26]:
#Checking for NA values
df.isna().sum()

Category    0
Detail      0
Cost        0
Date        0
dtype: int64

In [27]:
#Checking data types
df.dtypes

Category    object
Detail      object
Cost        object
Date        object
dtype: object

In [28]:
# Checking the shape of the dataframe
print("The dataframe has " +str(df.shape[0]) +" rows and " +str(df.shape[1])+ " columns.")

The dataframe has 329 rows and 4 columns.


In [29]:
# check for duplicate rows where every column value is duplicated
duplicate_rows = df[df.duplicated(subset=df.columns, keep=False)]

# print the duplicate rows
print(duplicate_rows)

Empty DataFrame
Columns: [Category, Detail, Cost, Date]
Index: []


In [30]:
# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [32]:
# Convert the 'cost' column to float
df['Cost'] = df['Cost'].str.replace('€', '').astype(float)

In [33]:
# Checking code worked
df.dtypes

Category            object
Detail              object
Cost               float64
Date        datetime64[ns]
dtype: object

In [35]:
# Checking the first and last day of the dataset
df.Date.min(),df.Date.max()

(Timestamp('2022-07-25 00:00:00'), Timestamp('2023-02-24 00:00:00'))

In [19]:
# Count of data points per month
df.groupby(df.Date.dt.month)['Category'].count()

Date
1     39
2     41
7     13
8     53
9     47
10    44
11    52
12    35
Name: Category, dtype: int64

In [60]:
# Defining the start and end dates for each period
periods = {'Jul - Aug 22': ('2022-07-25', '2022-08-24'),
           'Aug - Sep 22': ('2022-08-25', '2022-09-24'),
           'Sep - Oct 22': ('2022-09-25', '2022-10-24'),
           'Oct - Nov 22': ('2022-10-25', '2022-11-24'),
           'Nov - Dec 22': ('2022-11-25', '2022-12-24'),
           'Dec - Jan 22': ('2022-12-25', '2023-01-24'),
           'Jan - Feb 23': ('2023-01-25', '2023-02-24')}

# Sort the keys in chronological order
periods = dict(sorted(periods.items(), key=lambda x: pd.to_datetime(x[1][0])))

# Iterating over the periods and creating a new column with the corresponding period label
for period, dates in periods.items():
    start_date = pd.to_datetime(dates[0])
    end_date = pd.to_datetime(dates[1])
    mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
    df.loc[mask, 'Month_period'] = period

# Checking the new column
df.head()

,Category,Detail,Cost,Date,Month_period
0,Eating Out,Brunch,15.00,2022-07-25,Jul - Aug 22
1,Shopping,Uniqlo,144.50,2022-07-25,Jul - Aug 22
2,Hobbies,Salsa Class,130.00,2022-07-27,Jul - Aug 22
3,Groceries,Groceries,6.78,2022-07-27,Jul - Aug 22
4,Eating Out,Ditsch,2.00,2022-07-27,Jul - Aug 22


## Looking at the data

### Monthly Period vs Cost vs Category
Months started from the 25th of every month until the 24th of the following month, since pay day was always around or closest to the 25th.

In [61]:
# Grouping by month_period and summing the costs
month_period_costs = df.groupby('Month_period')['Cost'].sum().round(2)

# Sort the results by the chronological order of the periods
month_period_costs = month_period_costs.reindex(list(periods.keys()))

print(month_period_costs)

Month_period
Jul - Aug 22    1728.87
Aug - Sep 22    1776.55
Sep - Oct 22    1631.22
Oct - Nov 22    1098.53
Nov - Dec 22    1374.09
Dec - Jan 22    1178.47
Jan - Feb 23    1593.46
Name: Cost, dtype: float64


In [63]:
# Grouping by month_period and category and summing the costs, then sorting in chronological order
month_period_category_costs = df.groupby(['Month_period', 'Category'])['Cost'].sum().round(2)
month_period_category_costs = month_period_category_costs.reindex(pd.MultiIndex.from_product([periods.keys(), df['Category'].unique()], names=['Month_period', 'Category']))

print(month_period_category_costs)

Month_period  Category    
Jul - Aug 22  Eating Out      116.40
              Shopping        201.29
              Hobbies         174.34
              Groceries       362.07
              Rent + Bills    637.75
                               ...  
Jan - Feb 23  Transport       180.77
              Student Loan     82.43
              Travel          290.62
              Healthcare       67.11
              Selfcare           NaN
Name: Cost, Length: 77, dtype: float64


# Visualizations

**Bar chart**